In [1]:
import cv2
import numpy as np
import time
import threading
from queue import Queue

class RealTimeShapeTextExtractor:
    """
    ウェブカメラからのリアルタイム映像で概形と文字領域を検出するクラス
    """
    
    def __init__(self, camera_id=0, process_width=400, display_width=800):
        # カメラ設定
        self.camera_id = camera_id
        self.cap = None
        
        # 処理サイズ設定（処理は小さいサイズ、表示は大きいサイズ）
        self.process_width = process_width
        self.display_width = display_width
        
        # 検出パラメータ
        self.blur_ksize = (9, 9)
        self.canny_thresh1 = 50
        self.canny_thresh2 = 150
        self.min_text_area = 30
        self.max_text_area = 5000
        self.min_aspect_ratio = 0.1
        self.max_aspect_ratio = 8
        
        # 処理制御
        self.process_every_n_frames = 3  # N フレームごとに処理
        self.frame_count = 0
        self.last_detection_result = None
        
        # フレームレート計算用
        self.fps_counter = 0
        self.fps_start_time = time.time()
        self.current_fps = 0
        
        # 表示モード
        self.display_mode = 0  # 0: 原画像, 1: 概形, 2: 文字検出結果, 3: 二値化画像
        self.show_fps = True
        self.show_detection_info = True
        
        # 処理状態
        self.processing = False
        self.last_shape_mask = None
        
    def initialize_camera(self):
        """カメラの初期化"""
        self.cap = cv2.VideoCapture(self.camera_id)
        if not self.cap.isOpened():
            raise RuntimeError(f"カメラ {self.camera_id} を開けませんでした")
        
        # カメラ設定の最適化
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        self.cap.set(cv2.CAP_PROP_FPS, 30)
        self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)  # バッファサイズを小さくして遅延を減らす
        
        print(f"カメラ初期化完了:")
        print(f"  解像度: {int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))}x{int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))}")
        print(f"  FPS: {int(self.cap.get(cv2.CAP_PROP_FPS))}")
    
    def resize_frame(self, frame, target_width):
        """フレームをリサイズ"""
        h, w = frame.shape[:2]
        ratio = target_width / w
        new_height = int(h * ratio)
        return cv2.resize(frame, (target_width, new_height)), ratio
    
    def extract_shape_fast(self, frame):
        """高速な概形抽出"""
        # 処理用サイズにリサイズ
        small_frame, ratio = self.resize_frame(frame, self.process_width)
        gray = cv2.cvtColor(small_frame, cv2.COLOR_BGR2GRAY)
        
        # 高速ブラー
        blurred = cv2.GaussianBlur(gray, self.blur_ksize, 0)
        
        # Canny エッジ検出
        edges = cv2.Canny(blurred, self.canny_thresh1, self.canny_thresh2)
        
        # 軽量なモルフォロジー演算
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
        
        # 輪郭抽出
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        ###############################################################################ここまででデバッグ
        mask = np.zeros_like(gray)
        if contours:
            # 面積上位3つの輪郭を考慮（より安定した検出）
            contours = sorted(contours, key=cv2.contourArea, reverse=True)[:5]
            for cnt in contours:
                if cv2.contourArea(cnt) > 1000:  # 最小面積フィルタ
                    cv2.drawContours(mask, [cnt], -1, 255, -1)
        
        # 元のサイズにリサイズ
        h, w = frame.shape[:2]
        mask_resized = cv2.resize(mask, (w, h))
        shape_img = cv2.bitwise_and(frame, frame, mask=mask_resized)
        
        return shape_img, mask_resized, edges
    
    def extract_text_regions_fast(self, frame, shape_mask=None):
        """高速な文字領域抽出"""
        # 概形マスクがある場合は適用
        if shape_mask is not None:
            frame = cv2.bitwise_and(frame, frame, mask=shape_mask)
        
        # 処理用サイズにリサイズ
        small_frame, ratio = self.resize_frame(frame, self.process_width)
        gray = cv2.cvtColor(small_frame, cv2.COLOR_BGR2GRAY)
        
        # 高速二値化
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # 軽量なノイズ除去
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        
        # 輪郭抽出
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        text_boxes = []
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            area = w * h
            aspect_ratio = w / h if h > 0 else 0
            
            # 文字領域判定（より緩い条件で高速化）
            if (self.min_text_area < area < self.max_text_area and 
                h > 6 and w > 4 and 
                self.min_aspect_ratio < aspect_ratio < self.max_aspect_ratio):
                
                # 元のサイズに座標を変換
                x_orig = int(x / ratio)
                y_orig = int(y / ratio)
                w_orig = int(w / ratio)
                h_orig = int(h / ratio)
                
                text_boxes.append((x_orig, y_orig, w_orig, h_orig))
        
        return text_boxes, binary
    
    def draw_results(self, frame, text_boxes, fps, detection_count):
        """検出結果を描画"""
        result_frame = frame.copy()
        
        # 文字領域を描画
        for i, (x, y, w, h) in enumerate(text_boxes):
            cv2.rectangle(result_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            # テキストボックスの番号を表示（小さく）
            if len(text_boxes) < 20:  # 多すぎる場合は番号を省略
                cv2.putText(result_frame, f'{i+1}', (x, y-5), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
        
        # 情報表示
        if self.show_fps:
            cv2.putText(result_frame, f'FPS: {fps:.1f}', (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        if self.show_detection_info:
            cv2.putText(result_frame, f'Text Regions: {detection_count}', (10, 60), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        # 操作ガイド
        mode_names = ['Original', 'Shape', 'Detection', 'Binary']
        cv2.putText(result_frame, f'Mode: {mode_names[self.display_mode]} (Press 1-4)', 
                   (10, result_frame.shape[0] - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(result_frame, 'Press Q to quit, S to save, F to toggle FPS', 
                   (10, result_frame.shape[0] - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        return result_frame
    
    def calculate_fps(self):
        """FPS計算"""
        self.fps_counter += 1
        if self.fps_counter % 10 == 0:  # 10フレームごとに更新
            current_time = time.time()
            self.current_fps = 10 / (current_time - self.fps_start_time)
            self.fps_start_time = current_time
    
    def save_current_frame(self, frame, text_boxes):
        """現在のフレームを保存"""
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        
        # 検出結果付きの画像を保存
        result_frame = self.draw_results(frame, text_boxes, self.current_fps, len(text_boxes))
        filename = f"capture_{timestamp}.png"
        cv2.imwrite(filename, result_frame)
        
        print(f"フレームを保存しました: {filename}")
        print(f"検出された文字領域数: {len(text_boxes)}")
    
    def run(self):
        """メインループ"""
        try:
            self.initialize_camera()
            print("\nリアルタイム処理開始")
            print("操作方法:")
            print("  1-4キー: 表示モード切り替え")
            print("  Sキー: 現在のフレームを保存")
            print("  Fキー: FPS表示ON/OFF")
            print("  Qキー: 終了")
            print("-" * 50)
            
            while True:
                ret, frame = self.cap.read()
                if not ret:
                    print("フレーム読み取りエラー")
                    break
                
                # フレームカウント
                self.frame_count += 1
                self.calculate_fps()
                
                # 処理するフレームを間引き
                if self.frame_count % self.process_every_n_frames == 0:
                    self.processing = True
                    
                    # 概形抽出
                    shape_img, shape_mask, edges = self.extract_shape_fast(frame)
                    self.last_shape_mask = shape_mask
                    
                    # 文字領域抽出
                    text_boxes, binary = self.extract_text_regions_fast(frame, shape_mask)
                    
                    # 結果を保存
                    self.last_detection_result = {
                        'shape_img': shape_img,
                        'text_boxes': text_boxes,
                        'binary': binary,
                        'edges': edges
                    }
                    
                    self.processing = False
                
                # 表示フレーム準備
                display_frame, _ = self.resize_frame(frame, self.display_width)
                
                # 表示モードに応じて画像を選択
                if self.last_detection_result is not None:
                    result = self.last_detection_result
                    
                    if self.display_mode == 0:  # 原画像 + 検出結果
                        show_frame = self.draw_results(display_frame, 
                                                     self._scale_boxes(result['text_boxes'], 
                                                                     frame.shape[1], display_frame.shape[1]), 
                                                     self.current_fps, len(result['text_boxes']))
                    elif self.display_mode == 1:  # 概形
                        shape_resized, _ = self.resize_frame(result['shape_img'], self.display_width)
                        show_frame = self.draw_results(shape_resized, 
                                                     self._scale_boxes(result['text_boxes'], 
                                                                     frame.shape[1], shape_resized.shape[1]), 
                                                     self.current_fps, len(result['text_boxes']))
                    elif self.display_mode == 2:  # 検出結果のみ
                        show_frame = self.draw_results(display_frame, 
                                                     self._scale_boxes(result['text_boxes'], 
                                                                     frame.shape[1], display_frame.shape[1]), 
                                                     self.current_fps, len(result['text_boxes']))
                    elif self.display_mode == 3:  # 二値化画像
                        binary_resized = cv2.resize(result['binary'], (self.display_width, 
                                                   int(result['binary'].shape[0] * self.display_width / result['binary'].shape[1])))
                        show_frame = cv2.cvtColor(binary_resized, cv2.COLOR_GRAY2BGR)
                        if self.show_fps:
                            cv2.putText(show_frame, f'FPS: {self.current_fps:.1f}', (10, 30), 
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                else:
                    show_frame = display_frame
                
                # 画像表示
                cv2.imshow('Real-time Shape & Text Detection', show_frame)
                
                # キー入力処理
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q') or key == ord('Q'):
                    break
                elif key == ord('1'):
                    self.display_mode = 0
                elif key == ord('2'):
                    self.display_mode = 1
                elif key == ord('3'):
                    self.display_mode = 2
                elif key == ord('4'):
                    self.display_mode = 3
                elif key == ord('s') or key == ord('S'):
                    if self.last_detection_result is not None:
                        self.save_current_frame(frame, self.last_detection_result['text_boxes'])
                elif key == ord('f') or key == ord('F'):
                    self.show_fps = not self.show_fps
                elif key == ord('h') or key == ord('H'):
                    self.show_detection_info = not self.show_detection_info
        
        except KeyboardInterrupt:
            print("\n処理を中断しました")
        except Exception as e:
            print(f"エラーが発生しました: {e}")
        finally:
            self.cleanup()
    
    def _scale_boxes(self, boxes, original_width, target_width):
        """ボックス座標をスケール変換"""
        if not boxes:
            return []
        
        scale = target_width / original_width
        scaled_boxes = []
        for x, y, w, h in boxes:
            scaled_boxes.append((int(x * scale), int(y * scale), 
                               int(w * scale), int(h * scale)))
        return scaled_boxes
    
    def cleanup(self):
        """リソースの解放"""
        if self.cap is not None:
            self.cap.release()
        cv2.destroyAllWindows()
        print("リソースを解放しました")

class WebcamTextDetector:
    """簡単な使用のためのラッパークラス"""
    
    def __init__(self, camera_id=0):
        self.detector = RealTimeShapeTextExtractor(camera_id=camera_id)
    
    def start(self):
        """検出開始"""
        self.detector.run()
    
    def set_sensitivity(self, sensitivity='medium'):
        """検出感度の設定"""
        if sensitivity == 'high':
            self.detector.min_text_area = 20
            self.detector.max_text_area = 8000
            self.detector.process_every_n_frames = 2
        elif sensitivity == 'medium':
            self.detector.min_text_area = 30
            self.detector.max_text_area = 5000
            self.detector.process_every_n_frames = 3
        elif sensitivity == 'low':
            self.detector.min_text_area = 50
            self.detector.max_text_area = 3000
            self.detector.process_every_n_frames = 4

def main():
    """メイン関数"""
    print("=== リアルタイム文字領域検出システム ===")
    print("ウェブカメラを使用して文字領域をリアルタイム検出します")
    
    try:
        # 検出器の作成と開始
        detector = WebcamTextDetector(camera_id=0)  # 0 = デフォルトカメラ
        
        # 感度設定（オプション）
        detector.set_sensitivity('medium')  # 'high', 'medium', 'low'
        
        # 検出開始
        detector.start()
        
    except RuntimeError as e:
        print(f"カメラエラー: {e}")
        print("別のカメラIDを試してみてください（例: camera_id=1）")
    except Exception as e:
        print(f"予期しないエラー: {e}")

if __name__ == "__main__":
    main()


#ウィークポイントの検討
#環境条件のロバスト性


=== リアルタイム文字領域検出システム ===
ウェブカメラを使用して文字領域をリアルタイム検出します
カメラ初期化完了:
  解像度: 640x480
  FPS: 30

リアルタイム処理開始
操作方法:
  1-4キー: 表示モード切り替え
  Sキー: 現在のフレームを保存
  Fキー: FPS表示ON/OFF
  Qキー: 終了
--------------------------------------------------
リソースを解放しました


In [3]:
import cv2
import numpy as np
import json
import time

# --- 設定 ---
# ウェブカメラのID (通常は0)
WEBCAM_ID = 0
# フレーム処理の遅延時間 (ミリ秒) - リアルタイム性を調整
FRAME_DELAY_MS = 1
# 検出されたバウンディングボックスを描画する色 (BGR形式) - 今回はマスクに使用
BBOX_COLOR = (0, 255, 0) # 緑色
# バウンディングボックスの線の太さ
BBOX_THICKNESS = 2

# --- DBNetモデルの概念的なロードと初期化のプレースホルダー ---
# 実際のアプリケーションでは、ここにDBNetモデルをロードするコードが記述されます。
# 例:
# import torch
# import onnxruntime # ONNX形式のモデルを使用する場合
#
# try:
#     # 事前学習済みDBNetモデルのパスを指定
#     # このモデルファイルは、環境に事前に配置されている必要があります。
#     DBNET_MODEL_PATH = "path/to/your/dbnet_model.pth" # PyTorchモデルの場合
#     # DBNET_MODEL_PATH = "path/to/your/dbnet_model.onnx" # ONNXモデルの場合
#
#     # モデルのロード (例: PyTorch)
#     # dbnet_model = torch.load(DBNET_MODEL_PATH, map_location=torch.device('cpu'))
#     # dbnet_model.eval() # 推論モードに設定
#
#     # またはONNX Runtimeセッションの作成 (ONNXモデルの場合)
#     # sess_options = onnxruntime.SessionOptions()
#     # dbnet_session = onnxruntime.InferenceSession(DBNET_MODEL_PATH, sess_options)
#     # input_name = dbnet_session.get_inputs()[0].name
#     # output_name = dbnet_session.get_outputs()[0].name
#
#     print("DBNetモデルのロードを概念的に完了しました。")
# except Exception as e:
#     print(f"DBNetモデルのロード中にエラーが発生しました (シミュレーション): {e}")
#     # 実際のアプリケーションでは、ここでエラーハンドリングを行います。
#
# --- グローバルからローカルへの概念的な処理のためのプレースホルダー関数 ---
# 実際のシステムでは、この関数内で文書レイアウト解析（DLA）モデルが実行され、
# 文書の「概形」（例：列、テキストブロック、画像領域）を識別します。
# その後、simulate_dbnet_text_detection がこれらの概形領域内で詳細な文字を検出します。
def identify_global_document_shapes(frame):
    """
    フレームから主要な文書の「概形」を識別する概念的な関数。
    実際のアプリケーションでは、Recursive XY Cut (RXYC) や深層学習ベースの
    レイアウトセグメンテーションモデルがここに統合されます。
    このデモンストレーションでは、簡略化のためにフレーム全体を単一の「概形」として扱います。
    """
    # フレーム全体を単一のグローバル領域として返します。
    # 実際のDLAでは、複数の異なる領域（例：[x, y, w, h, 'text_column'], [x, y, w, h, 'image_area']）
    # が返されるでしょう。
    height, width, _ = frame.shape
    return [{'bbox': [0, 0, width, height], 'type': 'full_frame', 'id': 'global_0'}]

# --- 文字のような領域を検出する関数 (DBNetの動作をシミュレーション) ---
def simulate_dbnet_text_detection(frame, global_region):
    """
    指定されたグローバル領域内で文字のような領域を検出するシミュレーション関数。
    NOTE: 実際のアプリケーションでは、この関数はロードされたDBNetモデルを使用して
    テキスト検出推論を実行します。現在のOpenCVベースのロジックは、
    DBNetが達成するであろう「文字らしい領域の抽出」を概念的にシミュレートするものです。

    DBNetの一般的な処理フロー:
    1. 前処理: 入力画像をDBNetモデルの入力要件に合わせてリサイズ、正規化します。
    2. 推論: 前処理された画像をDBNetモデルに入力し、確率マップやバイナリマップなどの出力を取得します。
    3. 後処理: モデルの出力からテキスト領域のバウンディングボックス (またはポリゴン) を抽出します。
       これには、Differentiable Binarizationの逆操作や、非最大抑制 (NMS) などが含まれます。

    Args:
        frame (np.array): 入力画像フレーム。
        global_region (dict): 処理するグローバル領域のバウンディングボックス情報を含む辞書。

    Returns:
        list: 検出された文字のような領域のバウンディングボックスと信頼度スコアのリスト。
              例: [{'bbox': [x, y, w, h], 'confidence': 0.9}, ...]
    """
    # グローバル領域の座標を抽出
    x, y, w, h = global_region['bbox']
    
    # グローバル領域に基づいてフレームを切り抜き (実際のDLAではこの切り抜きが重要)
    cropped_frame = frame[y:y+h, x:x+w]
    if cropped_frame.shape[0] == 0 or cropped_frame.shape[1] == 0:
        return [] # 無効な切り抜きの場合は空リストを返す

    # --- ここからDBNetの「推論結果」をシミュレートするOpenCVロジック ---
    # 実際のDBNetでは、これらのOpenCV操作はDBNetモデルの内部で学習された特徴抽出と
    # テキスト領域のセグメンテーションに置き換えられます。
    
    # 1. グレースケールに変換
    gray = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2GRAY)

    # 2. 適応的二値化 (様々な照明条件に対応するため)
    # Otsu's binarization を使用してテキストと背景を分離
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # 3. モルフォロジー変換 (文字を結合して単語や行にする)
    # 適切なカーネルサイズはテキストのサイズと密度に依存します。
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    dilated = cv2.dilate(binary, kernel, iterations=1)

    # 4. 輪郭の検出
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected_regions = []
    for contour in contours:
        # 輪郭からバウンディングボックスを取得
        cx, cy, cw, ch = cv2.boundingRect(contour)

        # --- フィルタリング条件の強化 ---
        # これらの閾値は、検出したい文字のサイズと形状に合わせて調整する必要があります。
        # 小さすぎる領域や、極端なアスペクト比、ソリディティの低い領域を除外 (ノイズフィルタリング)
        
        # 輪郭の面積を計算
        area = cv2.contourArea(contour)
        
        # 最小面積の閾値 (調整)
        min_area = 100 
        if area < min_area:
            continue

        # アスペクト比のチェック (幅/高さ)
        min_aspect_ratio = 0.1
        max_aspect_ratio = 8.0 
        if ch > 0: # 高さで割る前にゼロ除算を避ける
            aspect_ratio = float(cw) / ch
            if not (min_aspect_ratio < aspect_ratio < max_aspect_ratio):
                continue

        # ソリディティのチェック (輪郭の面積 / その凸包の面積)
        hull = cv2.convexHull(contour)
        hull_area = cv2.contourArea(hull)
        if hull_area > 0: # ゼロ除算を避ける
            solidity = float(area) / hull_area
            min_solidity = 0.4 
            if solidity < min_solidity:
                continue
        else:
            continue # 凸包の面積が0の場合は無効な輪郭とみなす

        # --- フィルタリング条件の強化ここまで ---

        # グローバル領域に対する相対座標を元のフレーム座標に変換
        original_x = x + cx
        original_y = y + cy

        detected_regions.append({
            'bbox': [original_x, original_y, cw, ch],
            'confidence': 0.95 # シミュレーションのため固定値
        })
    
    return detected_regions

# --- メイン処理 ---
def main():
    cap = cv2.VideoCapture(WEBCAM_ID)

    if not cap.isOpened():
        print(f"エラー: ウェブカメラ (ID: {WEBCAM_ID}) を開けませんでした。")
        return

    print("ウェブカメラからのリアルタイム文字様領域抽出を開始します。")
    print(" 'q' キーを押すと終了します。")

    frame_count = 0
    start_time = time.time()

    while True:
        ret, frame = cap.read()
        if not ret:
            print("フレームを読み取れませんでした。終了します。")
            break

        frame_count += 1

        # 1. グローバルレイアウト解析 (概形認識)
        global_regions = identify_global_document_shapes(frame)
        
        all_detected_text_like_regions = []

        # 2. 各グローバル領域内でローカルな文字様要素を検出 (詳細情報認識)
        # ここで、DBNetモデルがロードされている場合、その推論が実行されます。
        for g_region in global_regions:
            # 実際のDBNet推論の呼び出しをシミュレート
            detected_local_regions = simulate_dbnet_text_detection(frame, g_region)
            all_detected_text_like_regions.extend(detected_local_regions)
            
        # 検出された文字様領域のみを抽出するためのマスクを作成
        height, width, _ = frame.shape
        mask = np.zeros((height, width, 3), dtype=np.uint8)

        # 検出された各文字様領域をマスク上で白く塗りつぶす
        for region in all_detected_text_like_regions:
            x, y, w, h = region['bbox']
            # 矩形領域を白 (255, 255, 255) で塗りつぶす
            cv2.rectangle(mask, (x, y), (x + w, y + h), (255, 255, 255), -1) # -1 は塗りつぶしを意味する

        # 元のフレームとマスクをAND演算して、文字様領域のみを抽出
        result_frame = cv2.bitwise_and(frame, mask)

        # フレームレートの計算と表示
        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time if elapsed_time > 0 else 0
        cv2.putText(result_frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        
        # 結果を表示
        cv2.imshow('リアルタイム文字様領域抽出 (黒塗り強調 - DBNet概念導入)', result_frame)

        # 'q' キーが押されたらループを終了
        if cv2.waitKey(FRAME_DELAY_MS) & 0xFF == ord('q'):
            break

    # リソースを解放
    cap.release()
    cv2.destroyAllWindows()
    print("プログラムを終了しました。")

if __name__ == "__main__":
    main()


ウェブカメラからのリアルタイム文字様領域抽出を開始します。
 'q' キーを押すと終了します。
プログラムを終了しました。
